In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy
import scipy
import math
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import sys
import os
#import socket
#import lmod
# lmod.load('system')
# lmod.load('texlive')
# lmod.
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib

In [5]:
n_cpus = 4
#print('** epoch: {}'.format(mpd.get_epoch()))
if 'SLURM_CPUS_PER_TASK' in os.environ.keys():
    n_cpus = int(os.environ['SLURM_CPUS_PER_TASK'])
#
print(f'*** n_cpus: {n_cpus}')


*** n_cpus: 4


In [35]:
SHP = hpc_lib.SH_PART_obj()

In [43]:
print('** ', SHP.cols)
print('** ', SHP.get_total_cpus('serc'), SHP.get_total('cpu_cores_total', 'serc'), SHP.get_total('cpu_cores_queued', 'serc') )
print('** ', SHP.get_total('cpu_cores_total','serc,normal'))
print('** ', SHP.get_total_cpus('serc,normal,hns'))
print('** ', SHP.get_total('gpus_queued', 'serc,hns'))

**  Index(['partition_name', 'partition_public', 'nodes_idle', 'nodes_total',
       'cpu_cores_idle', 'cpu_cores_total', 'cpu_cores_queued', 'gpus_idle',
       'gpus_total', 'gpus_queued', 'job_runtime_default',
       'job_runtime_maximum', 'mem/core_default', 'mem/core_maximum',
       'per-node_cores', 'per-node_mem(gb)', 'per-node_gpus'],
      dtype='object')
**  9104 9104 12422
**  13924
**  17652
**  72


In [37]:
print('** ', SHP.cols, SHP.SP.index)

**  Index(['partition_name', 'partition_public', 'nodes_idle', 'nodes_total',
       'cpu_cores_idle', 'cpu_cores_total', 'cpu_cores_queued', 'gpus_idle',
       'gpus_total', 'gpus_queued', 'job_runtime_default',
       'job_runtime_maximum', 'mem/core_default', 'mem/core_maximum',
       'per-node_cores', 'per-node_mem(gb)', 'per-node_gpus'],
      dtype='object') Index(['normal', 'bigmem', 'gpu', 'dev', 'hns', 'serc', 'owners'], dtype='object')


In [34]:
print('** ', SHP.cols)
print('** ', SHP.SP)

**  Index(['partition_name', 'partition_public', 'nodes_idle', 'nodes_total',
       'cpu_cores_idle', 'cpu_cores_total', 'cpu_cores_queued', 'gpus_idle',
       'gpus_total', 'gpus_queued', 'job_runtime_default',
       'job_runtime_maximum', 'mem/core_default', 'mem/core_maximum',
       'per-node_cores', 'per-node_mem(gb)', 'per-node_gpus'],
      dtype='object')
**         partition_name partition_public  nodes_idle  nodes_total  \
normal         normal              yes           0          179   
bigmem         bigmem              yes           1            9   
gpu               gpu              yes           0           26   
dev               dev              yes           3            4   
hns               hns               no           0          105   
serc             serc               no           0          233   
owners         owners               no          45         1467   

        cpu_cores_idle  cpu_cores_total  cpu_cores_queued  gpus_idle  \
normal            

In [ ]:
for rw in SHP.SP:
    print('** ', rw.split())

In [17]:
print('** ', len(b'abcd'))

**  4


In [ ]:
print('** ', SHP.sh_output_raw)

In [ ]:
sp_data = SHP.sh_output_raw
print('*** ', sp_data[0].split('|') )

In [ ]:
col_cats = [s.strip() for s in sp_data[0].split('|') if not s.strip()=='']
print(f'** col_cats: {col_cats}')

In [ ]:
col_groups = [[s.strip() for s in x.split()] for x in sp_data[1].split('|') if not x.strip()=='']
print('** ', col_groups)

In [ ]:
cols = [f'{s1}_{s2}'.strip().replace(' ', '_') for s1,S in zip(col_cats, col_groups) for s2 in S]
print(' ***\n', cols)